### TODO - CNN portion of project

1. Update metric to match those in paper (F1 score, precision, recall, AUC
2. Integrate phenotype dictionary to make selecting phenotype for Y-value in experiment easier (ie. not using a hard-coded integer)
3. Run experiments across all 10 phenotypes used in paper with default parameters
4. Repose this code
5. Add readMe
6. Create figure to compare F1 scores across phenotypes for the CNN

### Setup

In [1]:
# Mount into drive

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Project/src')

In [3]:
root = '/content/drive/MyDrive/Project'

In [4]:
import os
os.chdir(root)
%pwd

'/content/drive/.shortcut-targets-by-id/1AB0jDnt7V3OmP_E50BE3q1El8K68T-6-/Project'

In [5]:
# Detect PY file updates and reload
%load_ext autoreload
%autoreload 0.5

In [6]:
%ls

 BDH_reproducibility_challenge.pdf
 cse6250-project/
 CSE_6250_ProjectPaperSelection2.pdf
 CSE6250_ProjectPaperSelection.pdf
 CSE6250_ProjectProposal.pdf
 data/
 notebooks/
'Paper Notes.gdoc'
 Papers/
 ProjectNotes.gdoc
 ProjectTimeline.gsheet
 src/
'Team Registration & Paper Selection.gsheet'
 wandb/


In [7]:
%pwd

'/content/drive/.shortcut-targets-by-id/1AB0jDnt7V3OmP_E50BE3q1El8K68T-6-/Project'

### Installations

In [8]:
!pip install wandb -qqq

In [9]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alphabitten (cs7643-teamscam). Use `wandb login --relogin` to force relogin


True

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import logging
import time
import h5py
from platform import python_version
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam, Adadelta
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from platform import python_version
from torch.utils import data

# Local imports
import src.CNN.CNN_NLP as cnn_model
from src.CNN.data_load import get_data
from src.CNN.run_model import run_model

In [12]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: A100-SXM4-40GB


### Conditions Dictionary

In [13]:
conditions = {}
conditions[0] = 'cohort'
conditions[1] = 'Obesity'
conditions[2] = 'Non.Adherence'
conditions[3] = 'Developmental.Delay.Retardation'
conditions[4] = 'Advanced.Heart.Disease'
conditions[5] = 'Advanced.Lung.Disease'
conditions[6] = 'Schizophrenia.and.other.Psychiatric.Disorders'
conditions[7] = 'Alcohol.Abuse'
conditions[8] = 'Other.Substance.Abuse'
conditions[9] = 'Chronic.Pain.Fibromyalgia'
conditions[10] = 'Chronic.Neurological.Dystrophies'
conditions[11] = 'Advanced.Cancer'
conditions[12] = 'Depression'
conditions[13] = 'Dementia'
conditions[14] = 'Unsure'

### Experiment

In [14]:
import torch.optim as optim

phenotype = 1

sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'h5py_file': {
            'value':'src/phenotyping/their-embeddings/data-nobatch.h5'
        },
        'batch_size': {
            'values':[32, 64, 128]
        },
        'filter_sizes': {
            'value':[2, 3, 4, 5]
        },
        'num_filters': {
            'value':[100,100,100,100]
        },
        'num_classes': {
            'value':2
        },
        'dropout': {
            'values': [0.3, 0.5]
        },
        'learning_rate': {
            'values': [1e-1, 1e-2, 1e-3]
        },
        'phenotype_value': {
            'value': phenotype
        },
        'phenotype_description': {
            'value': conditions[phenotype]
        },
        'epochs': {
            'values':[100]
        },
        'opt': {
            'values':['ada']
        },
        'rho':{
            'values':[0.9, 0.95]
        },
        'freeze_embeddings':{
            'values':[True]
        }
    }
}

def run():
  with wandb.init(project="cs6250-project", entity="cs7643-teamscam") as run:
    config = wandb.config
    
    # Parameters
    H5PY_FILE = config["h5py_file"]
    BATCH_SIZE = config["batch_size"]
    FILTER_SIZES = config["filter_sizes"]
    NUM_FILTERS = config["num_filters"]
    NUM_CLASSES = config["num_classes"]
    DROPOUT = config["dropout"]
    LEARNING_RATE = config["learning_rate"]
    RHO = config["rho"]
    PHENOTYPE = config["phenotype_value"]
    EPOCHS = config["epochs"]
    FREEZE_EMBEDDINGS = config["freeze_embeddings"]

    # Get Train and Validation DataLoader
    train_dataloader, val_dataloader, embeddings_tensor = get_data(H5PY_FILE, device, BATCH_SIZE, PHENOTYPE)

    # Instantiate CNN model
    model = cnn_model.CNN_NLP(pretrained_embedding=embeddings_tensor,
                        freeze_embedding=FREEZE_EMBEDDINGS,
                        vocab_size=None,
                        embed_dim=300,
                        filter_sizes=FILTER_SIZES,
                        num_filters=NUM_FILTERS,
                        num_classes=NUM_CLASSES,
                        dropout=0.5)
    
    # Send model to `device` (GPU/CPU)
    model.to(device)
    
    # Instantiate Optimizer
    if (config['opt'] == 'adam'): 
      optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    else:
      optimizer = optim.Adadelta(model.parameters(), lr=LEARNING_RATE, rho=RHO)

    # Specify loss function
    loss_fn = nn.CrossEntropyLoss()

    # Instantiate the model run
    run = run_model(model, optimizer, loss_fn, device)

    # Run the train/validation
    results = run.train(train_dataloader, val_dataloader, EPOCHS)

count = 1  # number of runs to execute
sweep_id = wandb.sweep(sweep_config, project="cs6250-project", entity="cs7643-teamscam")
wandb.agent(sweep_id, function=run, count=count)




Create sweep with ID: 72sokxx6
Sweep URL: https://wandb.ai/cs7643-teamscam/cs6250-project/sweeps/72sokxx6


wandb: Agent Starting Run: 7gncof33 with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.5
wandb: 	epochs: 100
wandb: 	filter_sizes: [2, 3, 4, 5]
wandb: 	freeze_embeddings: True
wandb: 	h5py_file: src/phenotyping/their-embeddings/data-nobatch.h5
wandb: 	learning_rate: 0.1
wandb: 	num_classes: 2
wandb: 	num_filters: [100, 100, 100, 100]
wandb: 	opt: ada
wandb: 	phenotype_description: Obesity
wandb: 	phenotype_value: 1
wandb: 	rho: 0.9
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Start training...

 Epoch  |  Train Loss  |  Train Acc  |  Val Loss  |  Val Acc  |  Train F1   |  Val F1  |  Elapsed 
------------------------------------------------------------


  0%|          | 0/622 [00:00<?, ?it/s]

  0%|          | 0/89 [00:00<?, ?it/s]

...

  99    |   0.054021   |   97.99   |  0.289781  |   93.73   |   0.891494   |   0.583749   |   19.40  


  0%|          | 0/622 [00:00<?, ?it/s]

  0%|          | 0/89 [00:00<?, ?it/s]

  100   |   0.051139   |   98.13   |  0.252435  |   94.07   |   0.899019   |   0.614161   |   19.40  


Training complete! Best f1-score: 0.65%.


avg_train_f1,▁▁▃▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████████████
avg_train_loss,█▇▇▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
time_elapsed,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂▁▁▁
train_acc,▁▁▂▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████
train_accuracy,▁▁▂▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████
train_f1,▁▁▃▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████████████
train_loss,█▇▇▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▃▃▅▅▇▆▆▇▇▇▇▇▇▇▇▇▇▇██▇▇█████▇████▇▇▇▇▇█
val_f1,▁▁▄▄▅▅▇▆▆▇▇▇▇▇▇▇▇▇▇▇██▇██████▇█████▇████
val_loss,█▇▆▅▃▃▂▂▂▁▁▂▂▁▂▁▂▂▂▂▁▂▃▂▂▂▃▃▄▄▃▄▃▃▄▅▄▅▅▄
